254.736

# vns
Athletes: ['sun wei', 'zhang boheng', 'shi cong', 'weng hao']
 Score: 262.537

Hyper-Heuristics:

Athletes: ['sun wei' 'zhang boheng' 'shi cong' 'huang mingqi']  
Score: 259.90299999999996


Tabu search
Athletes:['weng hao' 'zhang boheng' 'sun wei' 'shi cong']
 Score:
262.537




In [41]:
import pandas as pd
import numpy as np

In [42]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')


In [43]:
df=df[df['Nation']=='China']


In [44]:
df=df.drop(columns=['Rank', 'Nation', 'round', 'year', 'AA'])

In [16]:
df

,Athlete,FX,PH,SR,VT,PB,HB
1,zhang boheng,14.133,14.666,14.866,14.766,15.300,14.166
3,shi cong,13.200,13.466,13.933,14.233,15.200,13.866
86,hu xuwei,0.000,0.000,0.000,0.000,15.233,14.533
131,weng hao,0.000,15.600,0.000,0.000,0.000,0.000
134,lan xingyu,0.000,0.000,15.266,0.000,0.000,0.000
175,huang mingqi,0.000,0.000,0.000,13.233,0.000,0.000
187,zhang boheng,14.883,13.466,14.600,14.866,15.366,14.800
192,shi cong,14.466,13.500,13.733,14.658,13.400,14.300
215,zhang boheng,14.500,14.000,12.033,13.600,14.900,14.733
320,you hao,0.000,13.633,14.633,0.000,12.700,13.233


In [17]:
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values
})


In [28]:
df = df_max_scores.copy()

In [29]:
import pandas as pd
import numpy as np
from itertools import combinations
from pulp import LpProblem, LpVariable, lpSum, LpMaximize


In [30]:
df['Athlete'].value_counts()

Athlete
hu xuwei        1
huang mingqi    1
lan xingyu      1
liu yang        1
shi cong        1
su weide        1
sun wei         1
weng hao        1
yang jiaxing    1
you hao         1
zhang boheng    1
zou jingyuan    1
Name: count, dtype: int64

# vns
Selected Athletes: ['sun wei', 'zhang boheng', 'shi cong', 'weng hao']
Total Score: 262.537

In [31]:
# VNS Algorithm with Iteration through random_state values
def variable_neighborhood_search(df):
    apparatus_order = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']  

    def calculate_total_score(athletes_df):
        # Sum the top 3 scores for each apparatus
        total_score = athletes_df[['FX', 'PH', 'SR', 'VT', 'PB', 'HB']].apply(lambda x: x.nlargest(3).sum()).sum()
        return total_score

    best_athletes = None
    best_score = float('-inf')
    best_seed = None

    for seed in range(6000): 
        # Randomly select 4 athletes with the current seed
        selected_athletes = df.sample(4, random_state=seed)

        # Check if each apparatus is competed on 3 times
        if all(selected_athletes[apparatus].count() >= 3 for apparatus in apparatus_order):
            # Calculate the total score for the current solution
            current_score = calculate_total_score(selected_athletes)

            # Compare with the current best solution
            if current_score > best_score:
                best_athletes = selected_athletes['Athlete'].tolist()
                best_score = current_score
                best_seed = seed

    return best_athletes, best_score, best_seed

# Run the VNS algorithm with different random_state values
best_athletes, total_score, best_seed = variable_neighborhood_search(df)

print("Selected Athletes:", best_athletes)
print("Total Score:", total_score)
print("Best Seed:", best_seed)


Selected Athletes: ['sun wei', 'zhang boheng', 'shi cong', 'weng hao']
Total Score: 262.537
Best Seed: 291


Hyper-Heuristics:


Best Selection: ['sun wei' 'zhang boheng' 'shi cong' 'huang mingqi']
Best Score: 259.90299999999996


In [32]:
df = df_max_scores.copy()

In [33]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['sun wei' 'zhang boheng' 'shi cong' 'huang mingqi']
Best Score: 259.90299999999996


TABU SEARCH
['weng hao' 'zhang boheng' 'sun wei' 'shi cong']
Total Score:
262.537




In [36]:
df = df_max_scores.copy()

In [37]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score



# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 246.76999999999998
Iteration 2, Best Score: 261.17
Iteration 3, Best Score: 262.537
Iteration 4, Best Score: 262.537
Iteration 5, Best Score: 262.537
Iteration 6, Best Score: 262.537
Iteration 7, Best Score: 262.537
Iteration 8, Best Score: 262.537
Iteration 9, Best Score: 262.537
Iteration 10, Best Score: 262.537
Iteration 11, Best Score: 262.537
Iteration 12, Best Score: 262.537
Iteration 13, Best Score: 262.537
Iteration 14, Best Score: 262.537
Iteration 15, Best Score: 262.537
Iteration 16, Best Score: 262.537
Iteration 17, Best Score: 262.537
Iteration 18, Best Score: 262.537
Iteration 19, Best Score: 262.537
Iteration 20, Best Score: 262.537
Iteration 21, Best Score: 262.537
Iteration 22, Best Score: 262.537
Iteration 23, Best Score: 262.537
Iteration 24, Best Score: 262.537
Iteration 25, Best Score: 262.537
Iteration 26, Best Score: 262.537
Iteration 27, Best Score: 262.537
Iteration 28, Best Score: 262.537
Iteration 29, Best Score: 262.537
Iteration 30,

In [39]:
import pandas as pd

data = {
    'Method': ['vns', 'Hyper-Heuristics', 'Tabu search'],
    'Athletes': [
        ['sun wei', 'zhang boheng', 'shi cong', 'weng hao'],
        ['sun wei', 'zhang boheng', 'shi cong', 'huang mingqi'],
        ['weng hao', 'zhang boheng', 'sun wei', 'shi cong']
    ],
    'Score': [262.537, 259.903, 262.537]
}

df = pd.DataFrame(data)
df


,Method,Athletes,Score
0,vns,"[sun wei, zhang boheng, shi cong, weng hao]",262.537
1,Hyper-Heuristics,"[sun wei, zhang boheng, shi cong, huang mingqi]",259.903
2,Tabu search,"[weng hao, zhang boheng, sun wei, shi cong]",262.537


In [40]:
# df.to_csv('china.csv', index=False)

In [78]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')
df=df[df['Nation']=='China']
df=df.drop(columns=['Rank', 'Nation', 'round', 'year'])

In [73]:
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values,
    'AA': df.loc[idx_max['AA'], 'AA'].values

})


In [74]:
df=df_max_scores.copy()

In [76]:
df=df.sort_values(by='AA', ascending=False)

In [77]:
df

,Athlete,VT,PH,SR,FX,PB,HB,AA
10,zhang boheng,14.900,14.666,14.866,14.883,15.366,14.800,87.981
4,shi cong,14.658,13.600,13.933,14.466,15.200,14.300,84.057
6,sun wei,14.200,12.966,14.266,13.600,14.400,14.833,82.698
9,you hao,0.000,13.633,14.800,0.000,14.233,13.233,54.199
8,yang jiaxing,14.466,12.666,0.000,13.300,0.000,13.666,54.098
5,su weide,14.266,10.900,0.000,14.200,0.000,14.300,53.666
3,liu yang,13.133,0.000,15.200,13.033,0.000,0.000,41.366
11,zou jingyuan,0.000,0.000,14.666,0.000,15.700,0.000,30.366
0,hu xuwei,0.000,0.000,0.000,0.000,15.233,14.533,29.766
7,weng hao,0.000,15.600,0.000,0.000,0.000,0.000,15.600


In [82]:
87.981+84.057+82.698

254.736